In [1]:
# Import pandas 
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm

In [2]:
# Load the test_results
test_results = pd.read_csv('datasets/AB_testing_results.csv',
                            parse_dates=True,
                            infer_datetime_format=True)


# Print the columns of test_results
print(test_results.columns)

Index(['uid', 'country', 'gender', 'spent', 'purchases', 'date', 'group',
       'device'],
      dtype='object')


### Confirming our test results
To begin this chapter, you will confirm that everything ran correctly for an A/B test similar to that shown in the lesson. Like the A/B test in the lesson this one consists of trying to boost consumable sales through making changes to a paywall.

The data from the test is loaded for you as "ab_test_results" and it has already been merged with the relevant demographics data. The checks you will perform will allow you to confidently report any results you uncover.

In [3]:
# Compute and print the results
results = test_results.groupby('group').agg({'uid': pd.Series.nunique}) 
print(results)

         uid
group       
GRP A  23009
GRP B  22874


In [4]:
# Find the overall number of unique users using "len" and "unique"
unique_users = len(test_results.uid.unique()) 

# Find the percentage in each group
results = results / unique_users * 100
print(results)

             uid
group           
GRP A  50.147113
GRP B  49.852887


In [5]:
# Find the unique users in each group, by device and gender
results = test_results.groupby(by=['group', 'device', 'gender']).agg({'uid': pd.Series.nunique}) 

# Find the overall number of unique users using "len" and "unique"
unique_users = len(test_results.uid.unique())

# Find the percentage in each group
results = results / unique_users * 100
print(results)

                           uid
group device gender           
GRP A A      F       12.239827
             M       12.795589
      I      F       12.542772
             M       12.568925
GRP B A      F       12.553669
             M       12.499183
      I      F       12.283417
             M       12.516618


### Intuition behind statistical significance
#### p-value:
In this exercise you will work to gain an intuitive understanding of statistical significance. You will do this by utilizing the get_pvalue() function on a variety of parameter sets that could reasonably arise or be chosen during the course of an A/B test. While doing this you should observing how statistical significance results vary as you change the parameters. This will help build your intuition surrounding this concept, and reveal some of the subtle pitfalls of p-values. As a reminder, this is the get_pvalue() function signature:

In [6]:
def get_pvalue(cont_conv, test_conv, cont_size, test_size):  
    lift =  - abs(test_conv - cont_conv)

    scale_one = cont_conv * (1 - cont_conv) * (1 / cont_size)
    scale_two = test_conv * (1 - test_conv) * (1 / test_size)
    scale_val = (scale_one + scale_two)**0.5

    p_value = 2 * stats.norm.cdf(lift, loc = 0, scale = scale_val )

    return p_value

In [7]:
# Get the p-value
p_value = get_pvalue(cont_conv=0.1, test_conv=0.17, cont_size=1000, test_size=1000)
print(p_value)

4.131297741047306e-06


In [8]:
# Get the p-value
p_value = get_pvalue(cont_conv=0.1, test_conv=0.15, cont_size=100, test_size=100)
print(p_value)

0.28366948940702086


In [9]:
# Get the p-value
p_value = get_pvalue(cont_conv=0.48, test_conv=0.50, cont_size=1000, test_size=1000)
print(p_value)

0.370901935824383


### Checking for statistical significance
Now that you have an intuitive understanding of statistical significance and p-values, you will apply it to your test result data.

The four parameters needed for the p-value function are the two conversion rates - cont_conv and test_conv and the two group sizes - cont_size and test_size. These are available in your workspace, so you have everything you need to check for statistical significance in our experiment results.

In [10]:
# Compute the p-value
p_value = get_pvalue(cont_conv=0.1, test_conv=0.17, cont_size=1000, test_size=1000)
print(p_value)

# Check for statistical significance
if p_value >= 0.05:
    print("Not Significant")
else:
    print("Significant Result")

4.131297741047306e-06
Significant Result


In [ ]:
# Calculate our test't power
get_power(test_size, cont_conv, test_conv, 0.95)

### Understanding confidence intervals
In this exercise, you'll develop your intuition for how various parameter values impact confidence intervals. Specifically, you will explore through the get_ci() function how changes widen or tighten the confidence interval. This is the function signature, where cl is the confidence level and sd is the standard deviation.

In [ ]:
def get_ci(value, cl, sd):
  loc = scipy.norm.ppf(1 - cl/2)
  rng_val = scipy.norm.cdf(loc - value/sd)

  lwr_bnd = value - rng_val
  upr_bnd = value + rng_val 

  return_val = (lwr_bnd, upr_bnd)
  return(return_val)

In [ ]:
# Compute and print the confidence interval.
# Find the confidence interval with a value of 1, a confidence level of 0.975 and a standard deviation of 0.5.
confidence_interval  = get_ci(1, 0.975, 0.5)
print(confidence_interval)

In [ ]:
# Compute and print the confidence interval
# Repeat the calculation, updating the confidence level to 0.95 and the standard deviation to 2. Leave the value as 1
confidence_interval  = get_ci(1, 0.95, 2)
print(confidence_interval)

In [ ]:
# Compute and print the confidence interval
# Finally, update your code such that the standard deviation is 0.001 while leaving the confidence level and value the same as the previous exercise part. 
# Compare the three confidence intervals outputted. How do they seem to relate to the parameters used?
confidence_interval  = get_ci(1, 0.95, 0.001)
print(confidence_interval)

#### Calculating confidence intervals
Now you will calculate the confidence intervals for the A/B test results.

The four values that have been calculated previously have been loaded for you (cont_conv, test_conv, test_size, cont_size) as variables with those names.

In [ ]:
def get_ci(test_conv, cont_conv, test_size, cont_size, ci):
    sd = ((test_conv * (1-test_conv)) / test_size +
         (cont_conv * (1-cont_conv)) / cont_size) **0.5
    lift = test_conv - cont_conv
    
    val = stats.norm.isf((1-ci)/2)
    lwr_bnd = lift - val * sd
    upr_bnd = lift + val * sd
    
    return((lwr_bnd, upr_bnd))

In [ ]:
# Calculate the mean of our lift distribution 
lift_mean = test_conv - cont_conv

# Calculate variance and standard deviation 
lift_variance = (1 - test_conv) * test_conv /test_size + (1 - cont_conv) * cont_conv / cont_size
lift_sd = lift_variance**0.5

# Find the confidence intervals with cl = 0.95
confidence_interval = get_ci(lift_mean, 0.95, lift_sd)
print(confidence_interval)

### Plotting the distribution
In this exercise, you will visualize the test and control conversion rates as distributions. It is helpful to practice what was covered in the example, as this may be something you have not applied before. Additionally, viewing the data in this way can give a sense of the variability inherent in our estimation.

Four variables, the test and control variances (test_var, cont_var), and the test and control conversion rates (test_conv and cont_conv) have been loaded for you.

In [ ]:
# Compute the standard deviations
control_sd = cont_var**0.5
test_sd = test_var**0.5

# Create the range of x values 
control_line = np.linspace(cont_conv - 3 * control_sd, cont_conv + 3 * control_sd, 100)
test_line = np.linspace(test_conv - 3 * test_sd, test_conv + 3 * test_sd, 100)

# Plot the distribution 
plt.plot(control_line, norm.pdf(control_line, cont_conv, control_sd))
plt.plot(test_line, norm.pdf(test_line, test_conv, test_sd))
plt.show()

### Plotting the difference distribution
Now lets plot the difference distribution of our results that is, the distribution of our lift.

The cont_var and test_var as well as the cont_conv and test_conv have been loaded for you. Additionally the upper and lower confidence interval bounds of this distribution have been provided as lwr_ci and upr_ci respectively.

In [ ]:
# Find the lift mean and standard deviation
lift_mean = test_conv - cont_conv
lift_sd = (test_var + cont_var) ** 0.5

# Generate the range of x-values
lift_line = np.linspace(lift_mean - 3 * lift_sd, lift_mean + 3 * lift_sd, 100)

# Plot the lift distribution
plt.plot(lift_line, norm.pdf(lift_line, lift_mean, lift_sd))

# Add the annotation lines
plt.axvline(x = lift_mean, color = 'green')
plt.axvline(x = lwr_ci, color = 'red')
plt.axvline(x = upr_ci, color = 'red')
plt.show()